# Работа с базами данных

## Задачи для совместного разбора

1. Работая с базой данных `Chinook_Sqlite.sqlite`, найдите и выведите на экран имена и фамилии всех заказчиков из Канады

In [17]:
import sqlite3

connection = sqlite3.connect('Chinook_Sqlite.sqlite')
cursor = connection.cursor()

cursor.execute("SELECT FirstName, LastName FROM Customer WHERE Country = 'Canada'")
customers = cursor.fetchall()

print("Customers from Canada:")
for customer in customers:
    print(customer[0], customer[1])


Customers from Canada:
François Tremblay
Mark Philips
Jennifer Peterson
Robert Brown
Edward Francis
Martha Silk
Aaron Mitchell
Ellie Sullivan


2. Найти и вывести на экран названия всех альбомов группы Accept

In [18]:
cursor.execute("SELECT DISTINCT Album.Title FROM Album JOIN Artist ON Album.ArtistId = Artist.ArtistId WHERE Artist.Name = 'Accept'")
albums = cursor.fetchall()

print("Albums by 'Accept':")
for album in albums:
    print(album[0])


Albums by 'Accept':
Balls to the Wall
Restless and Wild


3. Создайте базу данных с названием вашей группы. В этой базе данных создайте таблицу Student, содержащую 2 столбца: id и name. Добавьте в таблицу Student информацию о студентах, сидящих с вами по соседству.

In [19]:
conn = sqlite3.connect("our_group.db")
cursor = conn.cursor()

cursor.execute("""
    CREATE TABLE IF NOT EXISTS Student (
        id INTEGER PRIMARY KEY,
        name TEXT
    )
""")

students = [
    (1, 'zxc_0'),
    (2, 'zxc_1'),
    (3, 'zxc_2')
]

cursor.executemany("INSERT INTO Student (id, name) VALUES (?, ?)", students)

conn.commit()
conn.close()

## Лабораторная работа 6

1. Создайте файл базы данных c названием `recipes.db`. Создайте объект-курсор. 

In [34]:
conn = sqlite3.connect('recipes.db')
cursor = conn.cursor()

2. Напишите и выполните SQL-запрос для создания таблицы рецептов `Recipe`. Таблица должна содержать следующие поля:
`id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`. Определитесь с типами и составом ключевых полей.

In [35]:
cursor.execute('''
CREATE TABLE Recipe (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    minutes INTEGER,
    submitted TEXT,
    description TEXT,
    n_ingredients INTEGER
)
''')
conn.commit()
conn.close()

3. Напишите и выполните SQL-запрос для создания таблицы отзывов `Review`. Таблица должна содержать следующие поля:
`id`, `user_id`, `recipe_id`, `date`, `rating`, `review`. Определитесь с типами полей, набором ключевых полей. При помощи внешнего ключа соедините две таблицы.

In [36]:
conn = sqlite3.connect('recipes.db')
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE Review (
    id INTEGER PRIMARY KEY,
    user_id INTEGER,
    recipe_id INTEGER,
    date TEXT,
    rating INTEGER,
    review TEXT,
    FOREIGN KEY (recipe_id) REFERENCES Recipe(id)
)
''')

conn.commit()
conn.close()

4. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample_with_tags_ingredients.csv` (__ЛР5__) в созданные таблицы

In [37]:
def get_all():
    records = []
    cursor.execute('SELECT * FROM Review')
    rows = cursor.fetchall()
    for row in rows:
        records.append({'name': str(row[1]), 'id': str(row[0])})
    return records

conn = sqlite3.connect('recipes.db')
cursor = conn.cursor()

# print(get_all())

In [38]:
import csv

data_h = {}

tr = ['id', 'name', 'minutes', 'submitted', 'description', 'n_ingredients']
with open('recipes_sample_with_tags_ingredients.csv', newline='') as csvfile:
    reader = csv.reader(csvfile)
    h = next(reader)
    for row in reader:
        for i in range(len(h)):
            if h[i] not in data_h and h[i] in tr:
                data_h[h[i]] = []
                data_h[h[i]].append(row[i])
            elif h[i] in tr: data_h[h[i]].append(row[i])

In [39]:
import csv
import pandas as pd
conn = sqlite3.connect('recipes.db')
cursor = conn.cursor()

# with open('recipes_sample_with_tags_ingredients.csv', newline='') as csvfile:
#     reader = csv.reader(csvfile)
#     h = next(reader)
#     for row in reader:
#         for i in range(len(h)):
#             if h[i] in tr:
#                 cursor.execute('''INSERT INTO Recipe {h[i]} VALUES (?)''', row[i])
#     # reader = csv.reader(csvfile)
#     # next(reader)
#     # for row in reader:
#     #     cursor.execute('''INSERT INTO Recipe (id, name, minutes, submitted, description, n_ingredients)
#     #                       VALUES (?, ?, ?, ?, ?, ?)''', row)

# with open('reviews_sample.csv', newline='') as csvfile:
#     reader = csv.reader(csvfile)
#     next(reader)
#     for row in reader:
#         cursor.execute('''INSERT INTO Review (id, user_id, recipe_id, date, rating, review)
#                           VALUES (?, ?, ?, ?, ?, ?)''', row)

reviews_data = pd.read_csv("reviews_sample.csv")
reviews_data.to_sql('Review', conn, if_exists='replace', index=False)

recipes_data = pd.read_csv("recipes_sample_with_tags_ingredients.csv")
recipes_data.to_sql('Recipe', conn, if_exists='replace', index=False)

conn.commit()
conn.close()

5. Найдите все рецепты, для выполнения которых нужно ровно 10 ингредиентов. Выведите на экран первые 5 из найденных рецептов.

In [40]:
conn = sqlite3.connect('recipes.db')
cursor = conn.cursor()

query = "SELECT * FROM Recipe WHERE n_ingredients = 10"
cursor.execute(query)

results = cursor.fetchmany(5)
for result in results:
    print(result)

conn.close()


('blepandekager   danish   apple pancakes', 503475, 50, 128473, '2013-07-08', 10, "this recipe has been posted here for play in zwt9 - scandinavia.  this recipe was found at website: mindspring.com - christian's danish recipes.", 10, 1, 'european', 'milk*eggs*butter*flour*salt*cream*sugar*apples*cinnamon*lemon, juice of')
('kelly s creamy cheddar pea salad', 125195, 20, 113941, '2005-06-09', 5, "i'm not a big fan of peas, but like them in things. this salad is one of my favorites. happy to share!", 10, 1, '3-steps-or-less', 'onion*celery*cheddar cheese*mayonnaise*sugar*frozen peas*hard-boiled eggs*sweet pickle relish*seasoning salt*mustard')
('middle eastern   twice baked potatoes', 120297, 110, 197023, '2005-05-02', 14, 'a delicious yet low-fat twice-baked potato side dish with a blend of classic middle eastern flavours: chickpeas, cumin and coriander.  serve with roasts or with your favourite middle eastern main course dish.  adapted from "practical cookery: low fat".  these potatoes

6. Найдите название рецепта, для выполнения которого требуется больше всего времени.

In [41]:
conn = sqlite3.connect('recipes.db')
cursor = conn.cursor()

query = "SELECT name, minutes FROM Recipe ORDER BY minutes DESC LIMIT 1"
cursor.execute(query)

result = cursor.fetchone()

print("Recipe with the longest cooking time:", result)

conn.close()



Recipe with the longest cooking time: ('blueberry liqueur', 129615)


7. Запросите у пользователя id рецепта и верните информацию об этом рецепте. Если рецепт отсуствует, выведите соответствующее сообщение.

In [42]:
conn = sqlite3.connect('recipes.db')
cursor = conn.cursor()

recipe_id = 67664

query = "SELECT * FROM Recipe WHERE id = ?"
cursor.execute(query, (recipe_id,))

result = cursor.fetchone()

if result:
    print(result)
else:
    print("Recipe with id {} not found.".format(recipe_id))

conn.close()

('healthy for them  yogurt popsicles', 67664, 10, 91970, '2003-07-26', 1, 'my children and their friends ask for my homemade popsicles morning, noon and night. i never turn them down; who am i to tell them that they are good for them! for variety i substitute different flavours of frozen juice - grape, fruit punch, tropical etc.', 3, 1, '3-steps-or-less', 'milk*frozen juice concentrate*plain yogurt')


8. Найдите кол-во отзывов с рейтингом 5.

In [43]:
conn = sqlite3.connect('recipes.db')
cursor = conn.cursor()

print(cursor.execute("SELECT COUNT() FROM Review WHERE rating = 5").fetchone()[0])

conn.close()

91361


9. Найдите кол-во уникальных рецептов, не имеющих отзывов с рейтингом, меньше 4. 

In [45]:
conn = sqlite3.connect('recipes.db')
cursor = conn.cursor()

query = "SELECT COUNT() AS count FROM Recipe LEFT JOIN Review ON Recipe.id = Review.id WHERE Review.rating IS NULL OR Review.rating >= 4"

print(cursor.execute(query).fetchone()[0])

conn.close()

29597


10. Найдите кол-во рецептов, опубликованных в 2010 году и имеющих длину не менее 15 минут.

In [47]:
conn = sqlite3.connect('recipes.db')
cursor = conn.cursor()

query = "SELECT COUNT() FROM recipe WHERE strftime('%Y', submitted) = '2010' AND minutes >= 15"

print(cursor.execute(query).fetchone()[0])

conn.close()

1319


11. Выберите id рецепта, название рецепта, id пользователя, оставившего отзыв, дату отзыва и рейтинг для тех рецептов, которые имеют не менее 3 ингредиентов. Отсортируйте результат по id рецепта.

In [50]:
conn = sqlite3.connect('recipes.db')
cursor = conn.cursor()

# SELECT recipe.id, recipe.name, review.user_id, review.date, review.rating
# FROM recipe
# JOIN (
#     SELECT id, COUNT() AS ingredient_count
#     FROM recipe
#     GROUP BY id
#     HAVING ingredient_count >= 3
# ) AS ingredient_count_subquery ON recipe.id = ingredient_count_subquery.id
# JOIN review ON recipe.id = review.id
# ORDER BY recipe.id


query = "SELECT r.id AS recipe_id, r.name AS recipe_name, re.user_id, re.date, re.rating FROM recipe r JOIN (SELECT id, COUNT(*) AS num_ingredients FROM recipe GROUP BY id HAVING n_ingredients >= 3) ri ON r.id = ri.id JOIN review re ON r.id = re.id ORDER BY r.id"

print(cursor.execute(query).fetchall())

conn.close()

[(246, 'lee s hot crab dip', 487088, '2013-02-03', 5), (318, 'french pecan pie ii', 68460, '2003-12-20', 4), (321, 'drunken chicken ii', 149342, '2004-07-05', 3), (373, 'tiramisu   balducci s in new york city', 650039, '2008-01-27', 4), (387, 'chocolate coffee ice cream soda', 41578, '2005-02-12', 5), (445, 'remarkable rhubarb bites', 29655, '2010-06-02', 5), (659, 'drunken chicken marinade', 39334, '2008-02-10', 5), (1075, 'beef crumble', 362983, '2007-06-28', 5), (2560, 'confetti coleslaw', 1858347, '2014-01-27', 0), (2633, 'beef  pepper   mushroom kabobs', 341170, '2009-10-07', 3), (2704, 'banana sultana muffins', 587675, '2010-08-04', 5), (2883, 'carrot walnut cake', 128473, '2012-08-19', 5), (3212, 'pumpkin spice cake in jars', 178427, '2009-11-12', 5), (3252, 'crab boil spices', 75218, '2004-06-22', 5), (3616, 'oprah s classic mashed potatoes', 2597942, '2014-05-11', 4), (3653, 'eggplant  aubergine  and ziti parmesan', 31550, '2007-04-22', 5), (3657, 'general tso s chicken iii', 